In [ ]:
# Session identifier, do not remove or modify.
from yields.session import Session
session = Session("f1cfbf68-6aa4-47ea-a108-f231ce438970")

# Data quality analysis

In this notebook you will perform several data quality checks on a given dataset.

**Input**

* Dataset
* Data Model Mapping

**Output**

There are 2 outputs resulting from this notebook:

* A dictionary containing the DQ metrics
* An enriched dataset containing some additional outcomes of the DQ metrics, that can be further used in a downstream part using this enriched dataset. For example, we will add a column 'univariate_outlier' which takes a boolean value. This way, the DQ assessment results can be immediately used when one wants to assess the model performance on the inliers vs outliers.

## What are the 6 dimensions of data quality?

The following is taken from https://www.collibra.com/us/en/blog/the-6-dimensions-of-data-quality


* **Completeness**

Completeness measures if the data is sufficient to deliver meaningful inferences and decisions.<br>

This dimension can cover a variety of attributes depending on the entity. For customer data, it shows the minimum information essential for a productive engagement. For example, if the customer address includes an optional landmark attribute, data can be considered complete even when the landmark information is missing.<br>

For products or services, completeness can suggest vital attributes that help customers compare and choose. If a product description does not include any delivery estimate, it is not complete. Financial products often include historical performance details for customers to assess alignment with their requirements. 

* **Accuracy**

Data accuracy is the level to which data represents the real-world scenario and confirms with a verifiable source. Accuracy is very critical for highly regulated industries such as healthcare and finance. <br>

Accuracy of data ensures that the associated real-world entities can participate as planned. An accurate phone number of an employee guarantees that the employee is always reachable. Inaccurate birth details, on the other hand, can deprive the employee of certain benefits.<br>

* **Consistency**

This dimension represents if the same information stored and used at multiple instances matches. It is expressed as the percent of matched values across various records. Data consistency ensures that analytics correctly capture and leverage the value of data. <br>

Consistency is difficult to assess and requires planned testing across multiple data sets. If one enterprise system uses a customer phone number with international code separately, and another system uses prefixed international code, these formatting inconsistencies can be resolved quickly. However, if the underlying information itself is inconsistent, resolving may require verification with another source. For example, if a patient record puts the date of birth as May 1st, and another record shows it as June 1st, you may first need to assess the accuracy of data from both sources. *Data consistency is often associated with data accuracy, and any data set scoring high on both will be a high-quality data set.*

* **Validity**

This dimension signifies that the value attributes are available for aligning with the specific domain or requirement. For example, ZIP codes are valid if they contain the correct characters for the region. In a calendar, months are valid if they match the standard global names. *Using business rules is a systematic approach to assess the validity of data.* <br>

Any invalid data will affect the completeness of data. You can define rules to ignore or resolve the invalid data for ensuring completeness.

* **Uniqueness**

This dimension indicates if it is a single recorded instance in the data set used. Uniqueness is the most critical dimension for ensuring no duplication or overlaps. Data uniqueness is measured against all records within a data set or across data sets. A high uniqueness score assures minimized duplicates or overlaps, building trust in data and analysis.  


* **Integrity**

Data journey and transformation across systems can affect its attribute relationships. Integrity indicates that the attributes are maintained correctly, even as data gets stored and used in diverse systems. Data integrity ensures that all enterprise data can be traced and connected.<br>


While you regularly come across these six data quality dimensions, many more dimensions are available to represent distinctive attributes of data. Based on the context, you can also consider data conformity to standards (do data values comply with the specified formats?) for determining data qualityData quality is multi-dimensional and closely linked with data intelligence, representing how your organization understands and uses data.



<a id="toc"></a>

## Table of Contents
1. [Setup](#set)     

    1.1 [Import Libraries](#imp_lib)  
    
    1.2 [Load input](#load_input)   
    
2. [Uniqueness](#uniqueness)
                
3. [Completeness](#completeness)

4. [Validity](#validity)

    4.1 [Data type](#dt)
    
    4.2 [Range and constraint](#rac)
   
5. [Accuracy](#accuracy)

    5.1 [Consistency](#consistency)

    5.2 [Outliers](#outliers)
    
6. [Integrity](#integrity)

<a id="imp_lib"></a>
# [Import libraries](#toc)

In [ ]:
# We load platform related libraries as well as some general purpose libraries

from yields.providers.filesystem import ProviderFactoryFilesystem #To write into filesystem

# standard misc python libraries
import json, os
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.core.display import HTML

# plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#misc
from IPython.display import display, Markdown, Latex

# external library
from scipy import stats
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, activations, losses, optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Input, Flatten,\
                                    Reshape, LeakyReLU as LR,\
                                    Activation, Dropout
from tensorflow.keras.models import Model, Sequential
import scipy
import seaborn as sns
import collections

In [ ]:
# # User parameters
univariate_threshold = session.io.parameter('UNIVARIATE_OUTLIER_THRESHOLD').value # Threshold to determine outlier from inlier afer boxcox transformation of Mahalanobis distances


<a id="load_input"></a>
# [Load inputs](#toc)

In [ ]:
input_artifact_ds = session.io.input('ds')
file_handler = input_artifact_ds.file(input_artifact_ds.files[0]).open()
raw_data = pd.read_csv(file_handler)

input_artifact_dmm_ds = session.io.input('dmm_ds')
file_handler = input_artifact_dmm_ds.file(input_artifact_dmm_ds.files[0]).open()
schema = json.load(file_handler)

In [ ]:
raw_data.head()

In [ ]:
#As we will enrich the original dataset only with additional columns, it leaves the row_key intact. As the row_key is needed for saving (see last cell of this script) we will retrieve it here.
#If it is not a column of the dataframe itself, we will simply create an integer as row key.

# row_key = config._Config__config._inputs._data_sets['ds'].data_set._ingestion_config._row_key[0]

# if row_key in raw_data.columns:
#     row_key = row_key
# else:
#     raw_data = raw_data.reset_index()
#     row_key = 'index'

In [ ]:
# define an empty dictionary that will contain all the different metrics as we compute them along the line
data_quality_metrics = {}
data_quality_df = raw_data.copy()

<a id="uniqueness"></a>
# [Uniqueness](#toc)

Are there any duplicate rows or columns in the dataset?

In [ ]:
print("List of duplicate column NAMES: " + str([item for item, count in collections.Counter(raw_data.columns.tolist()).items() if count > 1]))

In [ ]:
# compute duplicate rows and columns 
remove_dup_rows = raw_data.drop_duplicates()
dup_rows = np.setdiff1d(raw_data.index.tolist(),remove_dup_rows.index.tolist()).tolist()

remove_dup_columns = raw_data.T.drop_duplicates().T
dup_cols = np.setdiff1d(raw_data.columns.tolist(),remove_dup_columns.columns.tolist()).tolist()

display(Markdown('- Duplicate rows: '+str(dup_rows)))
display(Markdown('- Duplicate columns: '+str(dup_cols)))

display(Markdown('- Number of duplicate rows: '+str(len(dup_rows))))
display(Markdown('- Number of duplicate columns: '+str(len(dup_cols))))

# update the y dict with the new computed metrics
data_quality_metrics.update({'List of duplicate rows': dup_rows})
data_quality_metrics.update({'List of duplicate columns':dup_cols})

<a id="completeness"></a>
# [Completeness](#toc)

Completeness measures if the data is sufficient to deliver meaningful inferences and decisions.

Here, we assess whether all the necessary data is present in a specific dataset. This means both of the following:

* Calculate the number of missing values per column. Empirically, 25% to 30% is the maximum missing values are allowed for columns, beyond which we might want to drop the column from analysis. However, this depends on the particular use case (for example, missing values might contain relevant information in predicting the quantity of interest).

* For each observation (row), calculate the number of missing values over all features.

* We also look for the categorical variables and count the number of categories present per variable. It is up to the user to compare this number with what is expected (or known). To approaches can be taken here (downstream part of this notebook)

    * Either some 'golden standard' is created that contains the known numbers for each category. For example, these numbers could be hard-coded in some dictionary which in turn can be uploaded as an artifact and read in this notebook. 
    
    * The number of categories on this dataset serve as a benchmark, and the dictionary that is saved here can be loaded in a downstream part. If then a new batch of data comes in, we can compare the number of categories in this new batch against the number of categories in this dataset. 
    
Note that this is a more direct manner to capture some 'novelty' or changes in the underlying process (as compared to more advanced techniques such as multivariate outlier detection).

In [ ]:
# extract the continous and categorical variables from the schema
cols = schema['continuous_inputs'] +   schema['categorical_inputs']

# compute the number of missing values per column and their percentages

df_completeness_col = pd.DataFrame(raw_data[cols].isnull().sum().sort_values(ascending=False), columns = ['Number missing'])
df_completeness_col['Percent missing'] = (df_completeness_col['Number missing']/raw_data.shape[0])

at_least_one_missing_value = df_completeness_col['Number missing'] > 0
if df_completeness_col[at_least_one_missing_value].shape[0] == 0:
    display(Markdown('<span style="color:green"> No missing values found.</span>'))
else:
    display(df_completeness_col[at_least_one_missing_value].style.format({'Percent missing': "{:.1%}"}).set_caption("Number of missing values (sum over all observations) per variable"))

In [ ]:
# compute the number of missing values per row, and their percentages
df_completeness_row = pd.DataFrame(raw_data[cols].isnull().sum(axis=1).sort_values(ascending=False), columns =['Number missing'])
df_completeness_row['Percent missing'] = (df_completeness_row['Number missing']/raw_data.shape[1])

at_least_one_missing_value = df_completeness_row['Number missing'] > 0
if df_completeness_row[at_least_one_missing_value].shape[0] == 0:
    display(Markdown('<span style="color:green"> No missing values found.</span>'))
else:
    display(df_completeness_row[at_least_one_missing_value].sample(5).style.format({'Percent missing': "{:.1%}"}).set_caption("Number of missing values (sum over all columns) for a few random observations (rows)"))

In [ ]:
display(Markdown('- Average percentage of missing values over all variables (columns): ' + str(df_completeness_col['Percent missing'].mean()*100)))
display(Markdown('- Average percentage of missing values per observation (row): '+ str(df_completeness_row['Percent missing'].mean()*100)))

# update the dictionary with the new computed metrics
data_quality_metrics.update({'Average percentage of missing values over all variables (columns)':df_completeness_col['Percent missing'].mean()})
data_quality_metrics.update({'Average percentage of missing values per observation (row)':df_completeness_row['Percent missing'].mean()})



Number of categories and the corresponding categories

In [ ]:
cat_vars = schema['categorical_inputs']

for col in cat_vars:
    
    raw_data_tmp = raw_data[col].dropna()
    cats , _ = np.unique(raw_data_tmp,return_counts = True)
    
    print(col + " : " + str(cats))
    

<a id="validity"></a>
# [Validity](#toc)

Data Validity can be defined as “the degree to which data values are consistent within a defined domain”. <br>
Using business rules is a systematic approach to assess the validity of data.<br>

In [ ]:
data_quality_metrics.update({'Number of invalid records': 'depends on use case'})

<a id="dt"></a>
## [Data type](#toc)

Exactly because the business rules are problem dependent, we only show here how to assess whether all continuous variables are numerical (e.g. no string type allowed for continuous variables). Other possible rules are:

* does a categorical field 'profession' only contain string type?
* if a variable denotes an index, we expect it to be of integer type
* Are all the 'amounts' in unit EURO / USD / ...?
* Are date(time) columns within given predefined set of possible signatures (e.g. all in yyyy-MM-dd or other)?
* ...

In [ ]:
# detect variable types
dtypes = ['int8', 'int16', 'int32', 'int64', 
          'uint8', 'uint16', 'uint32', 'uint64',
          'float_', 'float16', 'float32', 'float64']

# uncountable_var = np.array(schema.inputs)[[var in schema.uncountable for var in schema.inputs]]
df_uncountable = pd.DataFrame(raw_data[schema['continuous_inputs']].dtypes,columns=['Pandas data type'])
df_uncountable['Type is numeric in schema'] = [val not in dtypes for val in df_uncountable['Pandas data type']]

at_least_one_wrong_type = df_uncountable['Type is numeric in schema'] > 0
if at_least_one_wrong_type[at_least_one_wrong_type].shape[0] == 0:
    display(Markdown('<span style="color:green"> No uncountable variables with non-numeric data types.</span>'))
else:
    display(df_uncountable[df_uncountable['Type is numeric in schema']]);

In [ ]:
display(Markdown('- Number of uncountable variables with non-numeric data types: '+str(df_uncountable["Type is numeric in schema"].sum())))
data_quality_metrics.update({'Number of uncountable variables with non-numeric types': float(df_uncountable["Type is numeric in schema"].sum())})


<a id="rac"></a>
## [Range and constraints](#toc)

Here, another set of data-dependent business rules can be implemented, for example:

* variable 'age' should always be larger than or equal to zero (assessing whether the age itself corresponds with the true age of the observation is so-called data accuracy).
* Belgian postal code should always contain 4 digits.
* does the field 'password' meet the minimal length/complexity requirement?
* ...

In [ ]:
#Let's check variable age (as it is a continuous variable, we already know it's numeric, but we can go one step further and really check what it's type is.

# print('datatype of variable AGE:' + str(raw_data['AGE'].dtype))
# print('number of observations with negative age: ' + str(np.sum(np.where(raw_data['AGE']<0 , 1, 0))))

<a id="accuracy"></a>
# [Accuracy](#toc)

Data accuracy is the level to which data represents the real-world scenario and confirms with a verifiable source. <br>
It might seem similar to the data validity aspect, however, the difference can be illustrated by the following example showing that valid data can still be inaccurate:
the usage of valid postcode (that has passed technical and reference data validation) but that is used for the wrong suburb and thus ends up failing the data accuracy rules.<br>
Here, the combination of postcode + suburb does not exist in real life and thus is inaccurate. However, both might pass the validity tests as they both obey the given 'technical checks'.

Since the accuracy is use-case specific we leave it up to the user to implement his/her own analyses.

In [ ]:
data_quality_metrics.update({'Number of inaccurate records': 'depends on use case'})

<a id="consistency"></a>
## [Consistency](#toc)

This aspects represents if the same information is stored and used:

* across multiple datasources: as this requires the comparison of the same value of a variable between multiple datasets it is out-of-scope of this notebook.
* within a given dataset: here, we check for impossible combinations of values appearing by means of our own rules (i.e. rule-based). Examples are:
    - a date in one column, and the month corresponding to the date in another column. 
    - do the 'zip code' and the name of the province match?
    - 'age' should be compatible with year of birth (given the current date)
    - sum rules between different columns, e.g. given 'total maturity', 'remaining maturity' and 'time since beginning' we know that the sum of the two last should equal total maturity
    - 'begin date' < 'end date'

<a id="outliers"></a>
## [Outliers](#toc)

The above rule-based approach on searching for inaccurate records comes in handy when the user wants to be 100% certain that no such inaccurate records can occur. <br>
On the other hand, the search for such inaccurate observations can be automated by checking for (multivariate) outliers. This approach is different as it gives a list of *potentially* inaccurate records which in turn need manual inspection against the 'verifiable source' (as per definition of data accuracy).<br>

* We can look for outliers by only inspecting one variable at a time, i.e. so-called univariate outliers.

Let's take a bottom-up approach in terms of complexity (consider only continuous variables):

1. Simply describe your continuous data - inspecting the minima/maxima can already point out some inaccurate observations
2. Determine robust univariate outliers using the interquartile range - see https://en.wikipedia.org/wiki/Interquartile_range



### DESCRIBE

**CONTINUOUS VARIABLES**

In [ ]:
#1.Overview of the summary statistics of the continuous variables. Here we can already check some inaccurarcies e.g. by looking at the minima and maxima.
display(raw_data[schema['continuous_inputs']].describe().T)

**CATEGORICAL VARIABLES**

In [ ]:
#1.Overview of the summary statistics of the continuous variables. Here we can already check some inaccurarcies e.g. by looking at the minima and maxima.
display(raw_data[schema['categorical_inputs']].astype('object').describe().T)

### UNIVARIATE OUTLIERS

In [ ]:
#2. Use interquartile range to robustly detect univariate outliers (we assume symmetric distributions just to showcase).
#We return the list (index) of potential outliers and we also add aditional columns to the dataframe that indicate whether the observation is an outlier or not. This can be later used.

#Let's loop over all continuous variables as defined by the hybrid schema in the data model mapping

univariate_outliers = pd.DataFrame(columns = ['Number Univariate outliers', 'Percentage Univariate outliers'], index = schema['continuous_inputs'])

for col in schema['continuous_inputs']:
    
    eps = 1e-100
    x = raw_data[col]
    q1, q3 = np.quantile(x,0.25) , np.quantile(x,0.75)
    iq = q3 - q1
    lb , ub  = q1 - univariate_threshold*iq - eps,  q3 + univariate_threshold*iq + eps
    outliers_iq = np.where((x < lb) | (x > ub) , 1, 0)
    
    #Add numbers of outliers to the 'univariate_outliers' pandas dataframe
    univariate_outliers.loc[col, 'Number Univariate outliers'] = np.sum(outliers_iq)
    univariate_outliers.loc[col, 'Percentage Univariate outliers'] = np.sum(outliers_iq) / len(x)
    
    #Add the array outliers_iq (only containing 0/1) to our raw_dataframe
    data_quality_df = data_quality_df.join(pd.DataFrame(data = outliers_iq , index = raw_data.index, columns = [col + '_univariate_outlier']))

#Put the percentage into % formatting
univariate_outliers = univariate_outliers.style.format({'Percentage Univariate outliers': '{:.2%}'})
display(univariate_outliers)

In [ ]:
session.io.outputs

<a id="integrity"></a>
## [Integrity](#toc)

This aspects concerns that data is maintained properly within the institution across several datasources. It is therefore out-of-scope for the current notebook.

In [ ]:
output_artifact = session.io.output('ds_out')
artifact_output_version = output_artifact.file("test",ProviderFactoryFilesystem())
data_quality_df.to_csv(artifact_output_version, index=False) 
output_artifact.save()

# Save the data quality dictionary
output_artifact_2 = session.io.output('dq_report')
artifact_output_version = output_artifact_2.file("DQ_dictionary.json",ProviderFactoryFilesystem())
dq_report_to_json = json.dumps(data_quality_metrics)
artifact_output_version.write(dq_report_to_json.encode())
output_artifact_2.save()

Snapshot of the outlier enriched dataframe that will be saved

In [ ]:
display(data_quality_df.sample(5))

Data quality dictionary that will be saved:

In [ ]:
display(data_quality_metrics)

In [ ]:
print("end of notebook")